## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets.  

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session()

<refinitiv.data.session.Definition object at 0x1573c0419c0 {name='workspace'}>

In [2]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.1.0'

#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [3]:
# The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
response = search.Definition(
    top = 0,
    navigators = "RCSCountryLeaf(buckets:20)"   # Show the top 20
).get_data()
response.data.raw["Navigators"]["RCSCountryLeaf"]["Buckets"]

[{'Label': 'United States', 'Count': 13667077},
 {'Label': 'Eurobond', 'Count': 4559867},
 {'Label': 'China (Mainland)', 'Count': 1288801},
 {'Label': 'Colombia', 'Count': 1093363},
 {'Label': 'South Korea', 'Count': 583762},
 {'Label': 'Germany', 'Count': 506136},
 {'Label': 'Canada', 'Count': 496009},
 {'Label': 'France', 'Count': 273138},
 {'Label': 'India', 'Count': 212131},
 {'Label': 'Japan', 'Count': 165381},
 {'Label': 'Italy', 'Count': 135277},
 {'Label': 'Australia', 'Count': 134046},
 {'Label': 'Chile', 'Count': 131629},
 {'Label': 'Brazil', 'Count': 82689},
 {'Label': 'Switzerland', 'Count': 72062},
 {'Label': 'Austria', 'Count': 60897},
 {'Label': 'United Kingdom', 'Count': 60271},
 {'Label': 'Spain', 'Count': 59963},
 {'Label': 'Norway', 'Count': 58682},
 {'Label': 'Malaysia', 'Count': 56679}]

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [4]:
rd.discovery.search(
    view = search.Views.VESSEL_PHYSICAL_ASSETS,
    query = "tanker",
    select = "DocumentTitle, HullType",
    top = 10000
)

C:\Users\U8007876\AppData\Local\Programs\Python\Python310\lib\site-packages\refinitiv\data\content\search\_data_provider.py:309: UserWarning: Search result for tanker not full.
Result is maxed at 10000 while the total is 37371 rows.
Requested - 10000, skipped - 0 rows.
  warnings.warn(


,DocumentTitle,HullType
0,"M.T.THEA, Chemical Tanker, FAIRDEAL GROUP MANAGEMENT, English Channel|UK Continent Tanker Zone",Single Hull
1,"KK MARLIN, Chemical Tanker, South Africa|Western Cape|South Atlantic Ocean|South Africa Tanker Zone",<NA>
2,"HAFNIA NANJING, Jet Fuel; Chemical Tanker, MMS CO LTD, East Atlantic Ocean (Africa)|Gibraltar Tanker Zone",<NA>
3,"GW DOLPHIN, Chemical Tanker, English Channel|UK Continent Tanker Zone",<NA>
4,"MARITIME TRANQUILITY, Chemical Tanker, Caribbean Sea|Bahamas Tanker Zone|Dominican Republic",<NA>
...,...,...
9995,"RUBIKON9, Other Tanker, River Danube",<NA>
9996,"DONG MAO 9, Other Tanker, South China Sea|Canton Tanker Zone",<NA>
9997,"WEN JUN 2, Other Tanker, Shanghai Tanker Zone|Yangtze River",<NA>
9998,"EVGFDFGEYDJ.7, Other Tanker, East China Sea|Shanghai Tanker Zone",<NA>


Here, we can see a list of the different tankers and their Hull Types. Within this collection, I can clearly see there are different Hull Types list, i.e. "Single", "Double", "NA", etc, however, I'm also seeing duplicates.  In addition, I'm also exceeding the maximum limit of results I can retrieve and thus receive a warning.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [5]:
response=search.Definition(
    view = search.Views.VESSEL_PHYSICAL_ASSETS,
    query = "tanker",
    top = 0,       # I'm not interested in the hits, only details within the navigator response section
    navigators = "HullType"
).get_data()
response.data.raw["Navigators"]["HullType"]["Buckets"]

[{'Label': 'Double Hull', 'Count': 16552},
 {'Label': 'Single Hull', 'Count': 5419}]

#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [6]:
response=search.Definition(
    view = search.Views.EQUITY_QUOTES,
    filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'",    # Filter on all active (AC) assets within Canada
    top = 0,
    navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"  #  For the top 3 categories, show the top 2 exchanges
).get_data()

In [7]:
# Pretty display of the listing
rank = 1
for asset in response.data.raw["Navigators"]["RCSAssetCategoryLeaf"]["Buckets"]:
    print(f'{rank}. {asset["Label"]}')
    exchangeRank = 1
    for exchange in asset["ExchangeName"]["Buckets"]:
        print(f'\t{exchangeRank}. {exchange["Label"]} ({exchange["Count"]})')
        exchangeRank += 1
    rank += 1

1. Equity Cash Option
	1. Montreal Options Exchange (93063)
	2. Refinitiv (10679)
2. Ordinary Share
	1. Aequitas NEO-L (3569)
	2. Aequitas NEO-N (3568)
3. Equity Future Spread
	1. Montreal Exchange (6411)
	2. Montreal Options Exchange (36)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [8]:
# Get date N months ago
months = 3
date = dt.datetime.today() - dateutil.relativedelta.relativedelta(months=months)

In [9]:
# Retrieve the top 5 outstanding government debt in the last 3 months
response = search.Definition(
    view = search.Views.GOV_CORP_INSTRUMENTS,
    filter = "DbType eq 'govt' and IssueDate gt " + date.strftime("%Y-%m-%d"),
    top = 0,
    navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)"
).get_data()
response.data.raw["Navigators"]["RCSDomicileLeaf"]["Buckets"]

[{'Label': 'United States',
  'Count': 83,
  'max_CouponRate': 4.9360272,
  'sum_FaceOutstandingUSD': 2197069031300.0},
 {'Label': 'Japan',
  'Count': 51,
  'max_CouponRate': 1.6,
  'sum_FaceOutstandingUSD': 892720624360.0},
 {'Label': 'China (Mainland)',
  'Count': 38,
  'max_CouponRate': 2.8,
  'sum_FaceOutstandingUSD': 350631232878.0},
 {'Label': 'Singapore',
  'Count': 42,
  'max_CouponRate': 4.4944,
  'sum_FaceOutstandingUSD': 222540043497.0},
 {'Label': 'Hong Kong',
  'Count': 56,
  'max_CouponRate': 5.25,
  'sum_FaceOutstandingUSD': 143405148645.0}]

In [10]:
rd.close_session()